## **Phân tích dữ liệu thông minh - 20KHDL** ##
## **Lab Project - Predict Student Performance from Game Play** ##

---


### ***I. Giới thiệu thành viên và phân công công việc*** ###

|MSSV|Họ tên|Phân công công việc|
|-----|-----|------|
|20127136|Đặng Minh Đức|Quá trình khám phá dữ liệu + Quá trình tiền xử lý dữ liệu|
|20127571|Bùi Văn Nghĩa|b|
|20127676|Nguyễn Vũ Khôi|c|

### ***II. Giới thiệu chung về đồ án*** ###

### ***III. Code*** ###

#### Nạp các thư viện vào cho việc xử lý dữ liệu ####

In [1]:
import pandas as pd
import numpy as np

#### Quá trình khám phá dữ liệu ####

*1. Số dòng và số cột của train_label và test*

In [2]:
tmp = pd.read_csv("./train.csv",usecols=[0])
tmp = tmp.groupby('session_id').session_id.agg('count')

# COMPUTE READS AND SKIPS
PIECES = 10
CHUNK = int( np.ceil(len(tmp)/PIECES) )

reads = []
skips = [0]
for k in range(PIECES):
    a = k*CHUNK
    b = (k+1)*CHUNK
    if b>len(tmp): b=len(tmp)
    r = tmp.iloc[a:b].sum()
    reads.append(r)
    skips.append(skips[-1]+r)
    
print(f'To avoid memory error, we will read train in {PIECES} pieces of sizes:')
print(reads)

To avoid memory error, we will read train in 10 pieces of sizes:
[2684191, 2631991, 2638304, 2657670, 2644229, 2629801, 2596616, 2602258, 2619995, 2591891]


In [3]:
train = pd.DataFrame()

In [4]:
for i in range(PIECES):
    print(f'Piece {i+1} of {PIECES} with {reads[i]} reads and {skips[i]} skips')
    temp = pd.read_csv('./train.csv', skiprows=skips[i], nrows=reads[i])
    train = pd.concat([train, temp])

Piece 1 of 10 with 2684191 reads and 0 skips


Piece 2 of 10 with 2631991 reads and 2684191 skips
Piece 3 of 10 with 2638304 reads and 5316182 skips
Piece 4 of 10 with 2657670 reads and 7954486 skips
Piece 5 of 10 with 2644229 reads and 10612156 skips
Piece 6 of 10 with 2629801 reads and 13256385 skips
Piece 7 of 10 with 2596616 reads and 15886186 skips
Piece 8 of 10 with 2602258 reads and 18482802 skips
Piece 9 of 10 with 2619995 reads and 21085060 skips
Piece 10 of 10 with 2591891 reads and 23705055 skips


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26296946 entries, 0 to 2591890
Data columns (total 67 columns):
 #   Column                Dtype  
---  ------                -----  
 0   session_id            float64
 1   index                 float64
 2   elapsed_time          float64
 3   event_name            object 
 4   name                  object 
 5   level                 float64
 6   page                  float64
 7   room_coor_x           float64
 8   room_coor_y           float64
 9   screen_coor_x         float64
 10  screen_coor_y         float64
 11  hover_duration        float64
 12  text                  object 
 13  fqid                  object 
 14  room_fqid             object 
 15  text_fqid             object 
 16  fullscreen            float64
 17  hq                    float64
 18  music                 float64
 19  level_group           object 
 20  20110316334583480     float64
 21  1252                  float64
 22  1773951               float64
 23  checkpoint 

In [6]:
# Đọc file train_labels.csv (chỗ này sau khi xong hết ô giáo tổng hợp trên Kaggle và đọc file theo Kaggle + xóa cell (*) nha)
# do train.csv nó nặng qua, chạy banh cả màn hình nên copy 3353 dòng trong file train_reduce để chạy (mã số có thể sẽ ko giống vì khi copy nó hiện kiểu xE+0X)
train_df = pd.read_csv("/content/drive/MyDrive/20127136_IDA/Lab_Project/Data/train_reduce.csv", delimiter=',')
#train_labels_df = pd.read_csv("/content/drive/MyDrive/...", delimiter=',')
train_df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/20127136_IDA/Lab_Project/Data/train_reduce.csv'

In [ ]:
# Số dòng và số cột của test_df + train_label_df
print('Số dòng và số cột của train_label_df là ', train_labels_df.shape)
print('Số dòng và số cột của train_df là ', train_df.shape)

Số dòng và số cột của train_label_df là  (424116, 2)
Số dòng và số cột của train_df là  (4062, 20)


*2. Ý nghĩa của từng dòng và từng cột*

a. ``train_labels_df``

In [ ]:
# Trước hết chúng em sẽ xem tên của từng cột trong train_labels_df
train_labels_col_name = train_labels_df.columns

# In tên của cột trong train_labels_df
print('Các cột trong train_labels_df: ')
for i in range(0, len(train_labels_col_name)):
  print(i + 1, train_labels_col_name[i])

Các cột trong train_labels_df: 
1 session_id
2 correct


In [ ]:
# Trích 5 dòng của train_labels_df
train_labels_df.head(5)

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1


**Ý nghĩa của từng dòng trong dữ liệu**: Mỗi dòng của train_labels_df chứa thông tin về độ chính xác trong việc trả lời câu hỏi (với một số thứ tự câu hỏi nào đó) của một phiên. \

**Ý nghĩa của từng cột trong dữ liệu**: \

|Tên cột|Ý nghĩa|
|--------------------|--------------------------------------------------------------------------|
|`session_id`|Mã số ID của một session kèm theo câu hỏi mà học sinh trong session đó đã trả lời|
|`correct`|Kiểm tra độ chính xác trong việc trả lời câu hỏi của học sinh|

b. ``train_df``

In [ ]:
# Trước hết chúng em sẽ xem tên của từng cột trong test_labels_df
train_col_name = train_df.columns

# In tên của cột trong train_labels_df
print('Các cột trong train_df: ')
for i in range(0, len(train_col_name)):
  print(i + 1, train_col_name[i])

Các cột trong train_df: 
1 session_id
2 index
3 elapsed_time
4 event_name
5 name
6 level
7 page
8 room_coor_x
9 room_coor_y
10 screen_coor_x
11 screen_coor_y
12 hover_duration
13 text
14 fqid
15 room_fqid
16 text_fqid
17 fullscreen
18 hq
19 music
20 level_group


In [ ]:
# Trích 5 dòng của test_df
train_df.head(5)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,2.010000e+16,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,2.010000e+16,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,2.010000e+16,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,2.010000e+16,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,2.010000e+16,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


**Ý nghĩa của từng dòng trong dữ liệu**: Mỗi dòng của train_labels_df là thông tin của mỗi session trong trò chơi. Thông tin cụ thể nhóm em sẽ ghi trong phần
**Ý nghĩa của từng cột trong dữ liệu**.
**Ý nghĩa của từng cột trong dữ liệu**: \

|Tên cột|Ý nghĩa|
|--------------------|--------------------------------------------------------------------------|
|`session_id`|Mã số ID của một session|
|`index`|Số thứ tự của sự kiện cho session đó|
|`elapsed_time`|Thời gian từ lúc bắt đầu session đến lúc sự kiện được ghi nhận|
|`event_name`|Tên loại sự kiện|
|`name`|Tên sự kiện|
|`level`|Cấp độ mà sự kiện đạt đến trong game|
|`page`|Số trang của sự kiện|
|`room_coor_x`|Tọa độ trục hoành mà học sinh ấn chuột trong phòng chơi|
|`room_coor_y`|Tọa độ trục tung mà học sinh ấn chuột trong phòng chơi|
|`screen_coor_x`|Tọa độ trục hoành mà học sinh ấn chuột trên màn hình|
|`screen_coor_y`|Tọa độ trục tung mà học sinh ấn chuột trên màn hình|
|`hover_duration`|Thời gian di chuyển chuột (được tính bằng mi-li-giây)|
|`text`|Nội dung mà người chơi sẽ thấy trong quá trình sự kiện diễn tra trong trò chơi|
|`fqid`|Mã số ID đầy đủ của một sự kiện|
|`room_fqid`|Mã số ID đầy đủ của phòng mà học sinh nhấp chuột vào|
|`text_fqid`|Mã số ID đầy đủ của nội dung mà người chơi sẽ thấy trong quá trình sự kiện diễn tra trong trò chơi|
|`fullscreen`|Kiểm tra xem học sinh có để chế độ toàn màn hình hay không|
|`hq`|Kiểm tra chất lượng của trò chơi có cao hay không|
|`music`|Kiểm tra học sinh có bật nhạc trong quá trình chơi hay không|
|`level_group`|Phạm vi cấp độ - và phạm vi các câu hỏi mà học sinh đạt được|


*3. Kiểu dữ liệu của từng cột*

a. ``train_labels_df``

In [ ]:
# In ra kiểu dữ liệu của từng cột của train_labels_df
train_labels_df.dtypes

session_id    object
correct        int64
dtype: object

b. ``train_df``

In [ ]:
# In ra kiểu dữ liệu của từng cột của test_df
train_df.dtypes

session_id        float64
index               int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
dtype: object

**Nhận xét chung cho cả `train_labels_df` lẫn `train_df`**:
- Ở `train_labels_df` có `session_id` có kiểu dữ liệu là `object` và `correct` có kiểu dữ liệu là `int64` (kiểu số).
- Ở `train_df` có `event_name`, `name`, `text`, `fqiq`, `room_fqid`, `text_fqid` và `level_group` đều là kiểu dữ liệu `object`, còn lại là kiểu số (`int64` hay `float64`).
- Trên thực tế, nếu cột nào có kiểu dữ liệu `object` thì các phần tử của cột đó đều có một hoặc nhiều kiểu dữ liệu khác nhau. Để kiểm tra xem cột có kiểu dữ liệu `object` có hợp lý hay không, chúng em sẽ thực hiện code sau.

In [ ]:
# Kiểm tra xem cột có kiểu dữ liệu object (session_id) trong train_labels_df có hợp lý hay không
for col_name in train_labels_df.columns:
    col = train_labels_df[col_name]
    if col.dtype == 'object':
      dtype_set = set()
      train_labels_df[col_name].apply(lambda dt: dtype_set.add(type(dt)))
      print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'session_id' có thể có các kiểu dữ liệu: {<class 'str'>}


Từ đây, có thể thấy rằng kiểu dữ liệu `object` đối với cột `session_id` hợp lý vì cột này là mã số ID lẫn câu hỏi nên khi chúng em quan sát dữ liệu trước đó, chúng em nhận ra ngay rằng cột `session_id` phải là dạng chuỗi.

In [ ]:
# Kiểm tra xem cột có kiểu dữ liệu object (event_name, name, text, fqiq, room_fqid, text_fqid và level_group) trong train_df có hợp lý hay không
for col_name in train_df.columns:
    col = train_df[col_name]
    if col.dtype == 'object':
      dtype_set = set()
      train_df[col_name].apply(lambda dt: dtype_set.add(type(dt)))
      print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'event_name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text' có thể có các kiểu dữ liệu: {<class 'float'>, <class 'str'>}
Cột 'fqid' có thể có các kiểu dữ liệu: {<class 'float'>, <class 'str'>}
Cột 'room_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text_fqid' có thể có các kiểu dữ liệu: {<class 'float'>, <class 'str'>}
Cột 'level_group' có thể có các kiểu dữ liệu: {<class 'str'>}


Từ đây, có thể thấy rằng kiểu dữ liệu `object` đối với các cột `event_name`, `name`, `room_fqid` và `level_group` hợp lý vì những cột này là mã số ID hoặc tên nên khi chúng em quan sát dữ liệu trước đó, chúng em nhận ra ngay rằng cả bốn cột đều phải là dạng chuỗi. \
Còn kiểu dữ liệu `object` đối với các cột `text`, `fqid` và `text_fqid` không hợp lý vì như chúng em thấy các cột này đều có thể có 2 kiểu dữ liệu. Từ điều này chúng em sẽ tiền xử lý dữ liệu sao cho các cột đều có kiểu dữ liệu hợp lý.

*4. Kiểm tra sự bất thường trong dữ liệu* \
*Đây cũng chính là những cơ sở để nhóm em tiền xử lý dữ liệu.*

a. ``train_labels_df``

a.1. Kiểm tra dữ liệu bị trùng lặp

In [ ]:
# Kiểm tra xem có dòng nào bị trùng lặp trong train_labels_df hay không bằng cách sử dụng duplicated và lấy unique, chọn phần tử cuối cùng
train_labels_df.duplicated().unique()[-1]

False

Điều đó chứng tỏ dữ liệu không bị trùng lặp.

a.2. Kiểm tra cột bị thiếu dữ liệu

In [ ]:
# Kiểm tra xem có cột nào bị thiếu trong train_labels_df hay không bằng cách sử dụng info
train_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424116 entries, 0 to 424115
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   session_id  424116 non-null  object
 1   correct     424116 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.5+ MB


Sau quá trình quan sát thông tin trên, chúng em thấy rằng không có cột nào thiếu dữ liệu.

b. ``train_df``

b.1. Kiểm tra dữ liệu bị trùng lặp

In [ ]:
# Kiểm tra xem có dòng nào bị trùng lặp trong test_df hay không bằng cách sử dụng duplicated và lấy unique, chọn phần tử cuối cùng
train_df.duplicated().unique()[-1]

False

Điều đó chứng tỏ dữ liệu không bị trùng lặp.

b.2. Kiểm tra cột bị thiếu dữ liệu

In [ ]:
# Kiểm tra xem có cột nào bị thiếu trong test_df hay không bằng cách sử dụng info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4062 entries, 0 to 4061
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   session_id      4062 non-null   float64
 1   index           4062 non-null   int64  
 2   elapsed_time    4062 non-null   int64  
 3   event_name      4062 non-null   object 
 4   name            4062 non-null   object 
 5   level           4062 non-null   int64  
 6   page            87 non-null     float64
 7   room_coor_x     3537 non-null   float64
 8   room_coor_y     3537 non-null   float64
 9   screen_coor_x   3537 non-null   float64
 10  screen_coor_y   3537 non-null   float64
 11  hover_duration  515 non-null    float64
 12  text            1326 non-null   object 
 13  fqid            2848 non-null   object 
 14  room_fqid       4062 non-null   object 
 15  text_fqid       1326 non-null   object 
 16  fullscreen      4062 non-null   int64  
 17  hq              4062 non-null   i

Sau quá trình quan sát thông tin trên, chúng em thấy rằng các cột `page`, `room_coor_x`,`room_coor_y`, `screen_coor_x`, `screen_coor_y`, `hover_duration`, `text`, `fqid` và `text_fqid` thiếu dữ liệu, còn lại không thiếu.

#### Quá trình tiền xử lý dữ liệu ####

a. `train_labels_df`

Khi quan sát dữ liệu này, chúng em thấy rằng nếu muốn biết cụ thể session nào làm đúng câu nào thì không thể xác định được bằng cách xem cột `session_id` vì như vậy người đọc sẽ hiểu nhầm đó là mã số ID của session đó.  
Để có thể thực hiện việc này, mỗi dòng của cột `session_id`, chúng em sẽ tiền xử lý bằng cách tách chuỗi với chuỗi ngắt là `_q` và sau đó ta thêm vào 2 cột mới, 1 cột chứa id và 1 cột chứa số thứ tự câu hỏi.

In [ ]:
# Tách chuỗi cho từng dòng của cột session_id và sau đó ta thêm vào 2 cột mới
id_and_ques = train_labels_df['session_id'].str.split('_q', n=1, expand=True)
train_labels_df.insert(1, 's_id', id_and_ques[0])
train_labels_df.insert(2, 'q', id_and_ques[1])
train_labels_df['q'] = train_labels_df['q'].astype('int64') # Chuyển thành int64 với cột q

# In bảng ra
train_labels_df

,session_id,s_id,q,correct
0,20090312431273200_q1,20090312431273200,1,1
1,20090312433251036_q1,20090312433251036,1,0
2,20090312455206810_q1,20090312455206810,1,1
3,20090313091715820_q1,20090313091715820,1,0
4,20090313571836404_q1,20090313571836404,1,1
...,...,...,...,...
424111,22100215342220508_q18,22100215342220508,18,1
424112,22100215460321130_q18,22100215460321130,18,1
424113,22100217104993650_q18,22100217104993650,18,1
424114,22100219442786200_q18,22100219442786200,18,1


In [ ]:
# In ra sau khi tiền xử lý
train_labels_df

,session_id,s_id,q,correct
0,20090312431273200_q1,20090312431273200,1,1
1,20090312433251036_q1,20090312433251036,1,0
2,20090312455206810_q1,20090312455206810,1,1
3,20090313091715820_q1,20090313091715820,1,0
4,20090313571836404_q1,20090313571836404,1,1
...,...,...,...,...
424111,22100215342220508_q18,22100215342220508,18,1
424112,22100215460321130_q18,22100215460321130,18,1
424113,22100217104993650_q18,22100217104993650,18,1
424114,22100219442786200_q18,22100219442786200,18,1


Lúc bấy giờ, cột `session_id` trở nên vô nghĩa và chúng em sẽ tiến hành xóa cột đó. Sau đó đổi tên cột `s_id` thành `session_id` lại (vì trước khi xóa, khi thêm cột sau khi tách chuỗi, nếu đặt tên `session_id` thì sẽ bị trùng tên).

In [ ]:
# Xóa cột session_id, đổi tên cột s_id thành session_id và in ra sau khi xóa
train_labels_df = train_labels_df.drop(columns=['session_id'])
train_labels_df = train_labels_df.rename(columns={'s_id': 'session_id'})
train_labels_df

,session_id,q,correct
0,20090312431273200,1,1
1,20090312433251036,1,0
2,20090312455206810,1,1
3,20090313091715820,1,0
4,20090313571836404,1,1
...,...,...,...
424111,22100215342220508,18,1
424112,22100215460321130,18,1
424113,22100217104993650,18,1
424114,22100219442786200,18,1


b. `train_df`

Trước hết, vì chúng em thấy rằng cột ``page`` và cột ``text`` không mang lại ý nghĩa quan trọng cho việc xử lý nên chúng em sẽ tiến hành xóa hai cột này

In [ ]:
# Xóa cột page và text in ra sau khi xóa
train_df = train_df.drop(columns=['page', 'text'])
train_df

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,2.010000e+16,0,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,2.010000e+16,1,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,2.010000e+16,2,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,2.010000e+16,3,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,NaN,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,2.010000e+16,4,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,NaN,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4057,2.010010e+16,580,906242,cutscene_click,basic,13,-157.022938,-58.217144,358.0,359.0,NaN,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4058,2.010010e+16,581,907157,cutscene_click,basic,13,-157.022938,-58.217144,358.0,359.0,NaN,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4059,2.010010e+16,582,908003,cutscene_click,basic,13,-131.932029,-30.338356,376.0,339.0,NaN,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4060,2.010010e+16,583,909650,cutscene_click,basic,13,-131.932029,-30.338356,376.0,339.0,NaN,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22


Tiếp đến, chúng em sẽ tiền xử lý với những cột có dữ liệu bị thiếu, bao gồm các cột `room_coor_x`,`room_coor_y`, `screen_coor_x`, `screen_coor_y`, `hover_duration`, `fqid` và `text_fqid` bằng cách điền chúng. Chúng em đề xuất điền dữ liệu bị thiếu đối với các cột đó như sau: \

|Tên cột|Đề xuất|
|--------------------|--------------------------------------------------------------------------|
|`room_coor_x`|Điền tọa độ nhỏ nhất trong cột|
|`room_coor_y`|Điền tọa độ nhỏ nhất trong cột|
|`screen_coor_x`|Điền tọa độ nhỏ nhất trong cột|
|`screen_coor_y`|Điền tọa độ nhỏ nhất trong cột|
|`hover_duration`|Điền là -1 (để phân biệt thời gian xác định (>= 0) và thời gian không xác định)|
|`fqid`|Điền là `undefined_fqid`|
|`text_fqid`|Điền là `undefined_text_fqid`|



In [ ]:
# Điền dữ liệu bị thiếu đối với các cột room_coor_x,room_coor_y, screen_coor_x, screen_coor_y, hover_duration, fqid và text_fqid

train_df = train_df.fillna(value={'room_coor_x': train_df['room_coor_x'].min(), 'room_coor_y': train_df['room_coor_y'].min(),
                                  'screen_coor_x': train_df['screen_coor_x'].min(), 'screen_coor_y': train_df['screen_coor_y'].min(),
                                  'hover_duration': -1, 'fqid': 'undefined_fqid', 'text_fqid': 'undefined_text_fqid'})



In [ ]:
# In bảng sau khi điền dữ liệu bị thiếu
train_df

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,2.010000e+16,0,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,2.010000e+16,1,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,2.010000e+16,2,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,2.010000e+16,3,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,2.010000e+16,4,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4057,2.010010e+16,580,906242,cutscene_click,basic,13,-157.022938,-58.217144,358.0,359.0,-1.0,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4058,2.010010e+16,581,907157,cutscene_click,basic,13,-157.022938,-58.217144,358.0,359.0,-1.0,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4059,2.010010e+16,582,908003,cutscene_click,basic,13,-131.932029,-30.338356,376.0,339.0,-1.0,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22
4060,2.010010e+16,583,909650,cutscene_click,basic,13,-131.932029,-30.338356,376.0,339.0,-1.0,seescratches,tunic.historicalsociety.basement,tunic.historicalsociety.basement.seescratches,1,1,1,13-22


Đối với các cột có kiểu dữ liệu `object`, trong đó có thể có 2 kiểu dữ liệu như các cột `text`, `fqid` và `text_fqid`, ta sẽ chuyển kiểu dữ liệu `object` thành một kiểu dữ liệu thống nhất. Từ việc quan sát dữ liệu, chúng em nhận ra rằng cả ba cột này đều có dạng là chuỗi vì đó là những cột mang ý nghĩa là *mã ID và nội dung*. Sau đó khi in thông tin bằng hàm info, ta sẽ thấy nó vẫn là object nên chúng em sẽ lấy lại dòng code kiểm tra hợp lý của kiểu dữ liệu object ở phần III.3.b để xem nó đã thành kiểu dữ liệu `str` chưa.

In [ ]:
# Chuyển đổi kiểu dữ liệu sang str đối với các cột text, fqid và text_fqid
train_df['fqid'] = train_df['fqid'].astype('str')
train_df['text_fqid'] = train_df['text_fqid'].astype('str')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4062 entries, 0 to 4061
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   session_id      4062 non-null   float64
 1   index           4062 non-null   int64  
 2   elapsed_time    4062 non-null   int64  
 3   event_name      4062 non-null   object 
 4   name            4062 non-null   object 
 5   level           4062 non-null   int64  
 6   room_coor_x     4062 non-null   float64
 7   room_coor_y     4062 non-null   float64
 8   screen_coor_x   4062 non-null   float64
 9   screen_coor_y   4062 non-null   float64
 10  hover_duration  4062 non-null   float64
 11  fqid            4062 non-null   object 
 12  room_fqid       4062 non-null   object 
 13  text_fqid       4062 non-null   object 
 14  fullscreen      4062 non-null   int64  
 15  hq              4062 non-null   int64  
 16  music           4062 non-null   int64  
 17  level_group     4062 non-null   o

In [ ]:
# Kiểm tra xem kiểu dữ liệu của các cột fqid và cột text_fqid được thống nhất là str chưa
for col_name in train_df.columns:
  col = train_df[col_name]
  if col.dtype == 'object':
    dtype_set = set()
    train_df[col_name].apply(lambda dt: dtype_set.add(type(dt)))
    print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'event_name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'room_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'level_group' có thể có các kiểu dữ liệu: {<class 'str'>}


#### Quá trình tạo đặc trưng ####

Đối với những cột nào có kiểu dữ liệu là string chứa từ 2-4 giá trị riêng biệt, chúng em sẽ tạo đặc trưng bằng cách sử dụng *one-hot encoding*. Để biết cột nào có điều kiện như vậy thì chúng em sẽ chạy code sau:

In [ ]:
# In ra mỗi cột chứa bao nhiêu giá trị riêng biệt
for col in train_df.columns:
  if train_df[col].dtype == 'object':
    print(col, ': ', len(train_df[col].unique()))

event_name :  11
name :  6
fqid :  110
room_fqid :  19
text_fqid :  91
level_group :  3


Từ đây, chúng em thấy rằng cần tạo đặc trưng từ cột ``level_group``.

In [ ]:
# One-hot encoding đối với cột level_group và in bảng ra sau khi có các cột one-hot encoding
level_group_col = train_df['level_group']
level_group_col = pd.get_dummies(level_group_col, prefix='level_group', dtype=int) # one-hot encoding bằng cách sử dụng hàm get_dummies cho việc đổi từ chuỗi sang 0 hoặc 1 (giá trị số)
train_df = pd.concat([train_df, level_group_col], axis=1)
train_df

#### Xây dựng mô hình ####

#### Thử nghiệm ####

#### Kết luận ####

#### Hướng phát triển ####